# Class-Conditional TTA-NRAM Example

## 🎯 Key Innovation: Separate NRAM for Real and Fake

**Why Class-Conditional?**
- ✅ **Real images**: Natural textures, specific statistical properties
- ✅ **Fake images**: GAN artifacts, upsampling traces, different statistics  
- ❌ **Problem**: Mixing them causes incorrect variance estimation
- ✅ **Solution**: Separate NRAM modules optimized for each class

**Architecture:**
```
Input Image [B, 3, H, W]
    ↓
Base Model (frozen) → layer4 features
    ↓
Pseudo-Label Generation (quick prediction)
    ↓
┌─────────────────┬─────────────────┐
│   Real NRAM     │   Fake NRAM     │
│  (for real)     │  (for fake)     │
└─────────────────┴─────────────────┘
    ↓
Merged Features → Base Classifier
```

**Expected Improvement over Unified NRAM:**
- Clean data: +1-2%
- Noisy data: +5-10%

## 1. Import

In [ ]:
import sys
# Clear cache
for mod in list(sys.modules.keys()):
    if any(x in mod for x in ['NPR', 'npr', 'LGrad', 'lgrad', 'tta_nram']):
        del sys.modules[mod]

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms

# Dataset and metrics
from utils.data.dataset import CorruptedDataset
from utils.eval.metrics import PredictionCollector, MetricsCalculator

# Models
from model.LGrad.lgrad_model import LGrad
from model.NPR.npr_model import NPR

# Class-Conditional TTA-NRAM
from model.method.tta_nram import (
    ClassConditionalTTANRAM,
    TTANRAMConfig,
    inference_with_tta_conditional,
)

## 2. Configuration

In [ ]:
# Device
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# Model selection
MODEL = "LGrad"  # or "NPR"

# Datasets and corruptions
DATASETS = [
    "corrupted_test_data_progan",
    "corrupted_test_data_stylegan",
]

CORRUPTIONS = [
    "original",
    "gaussian_noise",
    "jpeg_compression",
]

# Paths
DATA_ROOT = "corrupted_dataset"

# TTA config
TTA_STEPS = 5
TTA_LR = 1e-4
BATCH_SIZE = 16

## 3. Load Base Model

In [ ]:
if MODEL == "LGrad":
    STYLEGAN_WEIGHTS = "model/LGrad/weights/karras2019stylegan-bedrooms-256x256_discriminator.pth"
    CLASSIFIER_WEIGHTS = "model/LGrad/weights/LGrad-Pretrained-Model/LGrad-4class-Trainon-Progan_car_cat_chair_horse.pth"
    
    base_model = LGrad(
        stylegan_weights=STYLEGAN_WEIGHTS,
        classifier_weights=CLASSIFIER_WEIGHTS,
        device=DEVICE
    )
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
    ])
    
elif MODEL == "NPR":
    NPR_WEIGHTS = "model/NPR/weights/NPR.pth"
    
    base_model = NPR(
        weights=NPR_WEIGHTS,
        device=DEVICE
    )
    
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

print(f"✅ {MODEL} base model loaded")

## 4. Create Class-Conditional TTA-NRAM Model

In [ ]:
# Config
config = TTANRAMConfig(
    model=MODEL,
    target_layer=None,  # Auto-detect
    reduction_ratio=16,
    
    # Noise estimation
    noise_detection_method="laplacian",
    noise_normalize_factor=100.0,
    
    # Memory bank (class-specific)
    enable_memory_bank=True,
    memory_size=100,
    confidence_threshold=0.8,
    
    # TTA settings
    tta_steps=TTA_STEPS,
    tta_lr=TTA_LR,
    tta_loss_weights={"entropy": 1.0, "confidence": 0.1},
    
    # Gating
    residual_weight=0.1,
    
    device=DEVICE
)

# Create class-conditional model
tta_model = ClassConditionalTTANRAM(base_model, config)

print("✅ ClassConditionalTTANRAM created")
print(f"   - Real NRAM: {sum(p.numel() for p in tta_model.real_nram.parameters()):,} params")
print(f"   - Fake NRAM: {sum(p.numel() for p in tta_model.fake_nram.parameters()):,} params")
print(f"   - Total trainable: {sum(p.numel() for p in tta_model.parameters() if p.requires_grad):,} params")

## 5. Load Dataset

In [ ]:
dataset = CorruptedDataset(
    root=DATA_ROOT,
    datasets=DATASETS,
    corruptions=CORRUPTIONS,
    transform=transform
)

print(f"Total samples: {len(dataset)}")
print(f"Datasets: {DATASETS}")
print(f"Corruptions: {CORRUPTIONS}")

## 6. Test on Single Batch

Let's see pseudo-label routing in action!

In [ ]:
# Get a batch with mixed real/fake
noisy_indices = [
    i for i, s in enumerate(dataset.samples)
    if s['dataset'] == "corrupted_test_data_progan" and s['corruption'] == "gaussian_noise"
]

test_loader = DataLoader(
    Subset(dataset, noisy_indices[:32]),
    batch_size=32,
    shuffle=False
)

batch = next(iter(test_loader))
images, labels, metadata = batch
images = images.to(DEVICE)
labels = labels.to(DEVICE)

print(f"Batch size: {images.shape[0]}")
print(f"Ground truth: Real={( labels==0).sum().item()}, Fake={(labels==1).sum().item()}")

In [ ]:
# TTA with class-conditional routing
tta_model.reset_memory()

print("Running Class-Conditional TTA...")
results = inference_with_tta_conditional(
    model=tta_model,
    images=images,
    config=config,
    return_debug=True
)

# Results
print("\n" + "="*60)
print("Class-Conditional TTA Results")
print("="*60)
print(f"Initial predictions (mean): {results['initial_predictions'].mean().item():.4f}")
print(f"Final predictions (mean):   {results['predictions'].mean().item():.4f}")
print(f"Improvement:                {results['improvement']:.4f}")

# Pseudo-labels
pseudo_labels = results['pseudo_labels']
print(f"\nPseudo-labels: Real={( pseudo_labels==0).sum().item()}, Fake={(pseudo_labels==1).sum().item()}")
print(f"Ground truth:  Real={( labels.cpu()==0).sum().item()}, Fake={(labels.cpu()==1).sum().item()}")

# Pseudo-label accuracy
pseudo_acc = (pseudo_labels == labels.cpu()).float().mean().item()
print(f"Pseudo-label accuracy: {pseudo_acc*100:.2f}%")

# TTA history
print("\nTTA History (class-wise):")
for step_info in results['tta_history']:
    print(f"  Step {step_info['step']}: loss={step_info['loss']:.4f}, "
          f"real={step_info.get('n_real', 0)}, fake={step_info.get('n_fake', 0)}")

# Final accuracy
preds = (results['predictions'] > 0.5).float().squeeze()
acc = (preds == labels.cpu().float()).float().mean().item()
print(f"\nFinal Accuracy: {acc*100:.2f}%")

## 7. Full Evaluation

Evaluate on all dataset-corruption combinations.

In [ ]:
def evaluate_class_conditional(model, dataloader, config, device, name="test"):
    """
    Evaluate ClassConditionalTTANRAM on dataloader.
    """
    model.eval()
    collector = PredictionCollector()
    calc = MetricsCalculator()
    
    # Reset memory
    model.reset_memory()
    
    # Track pseudo-label accuracy
    pseudo_correct = 0
    pseudo_total = 0
    
    pbar = tqdm(dataloader, desc=name)
    for batch in pbar:
        images, labels, metadata = batch
        images = images.to(device)
        
        # Class-conditional TTA
        results = inference_with_tta_conditional(
            model=model,
            images=images,
            config=config,
            return_debug=False
        )
        
        # Collect predictions
        probs = results['predictions']
        collector.update(labels, probs, threshold=0.5)
        
        # Track pseudo-label accuracy
        if results['pseudo_labels'] is not None:
            pseudo_labels = results['pseudo_labels']
            pseudo_correct += (pseudo_labels == labels).sum().item()
            pseudo_total += len(labels)
    
    # Compute metrics
    metrics = calc.compute_from_collector(collector, name=name)
    
    # Add pseudo-label accuracy
    if pseudo_total > 0:
        metrics['pseudo_label_acc'] = pseudo_correct / pseudo_total
    
    return metrics

In [ ]:
# Evaluate on all combinations
all_results = {}

for dataset_name in DATASETS:
    for corruption in CORRUPTIONS:
        indices = [
            i for i, s in enumerate(dataset.samples)
            if s['dataset'] == dataset_name and s['corruption'] == corruption
        ]
        
        if len(indices) == 0:
            continue
        
        print(f"\n{'='*60}")
        print(f"Evaluating: {dataset_name}-{corruption}")
        print(f"Samples: {len(indices)}")
        print(f"{'='*60}")
        
        dataloader = DataLoader(
            Subset(dataset, indices),
            batch_size=BATCH_SIZE,
            shuffle=False,
            num_workers=4,
            drop_last=True
        )
        
        # Evaluate
        metrics = evaluate_class_conditional(
            model=tta_model,
            dataloader=dataloader,
            config=config,
            device=DEVICE,
            name=f"{dataset_name}-{corruption}"
        )
        
        # Print results
        print(f"\nResults:")
        print(f"  Accuracy:          {metrics['accuracy']*100:.2f}%")
        print(f"  AUC:               {metrics['auc']*100:.2f}%")
        print(f"  F1:                {metrics['f1']*100:.2f}%")
        print(f"  Pseudo-label acc:  {metrics.get('pseudo_label_acc', 0)*100:.2f}%")
        
        all_results[(dataset_name, corruption)] = metrics

## 8. Summary

Class-conditional TTA-NRAM improves robustness by:
1. **Separating Real and Fake processing** (different feature statistics)
2. **Pseudo-label based routing** (automatic classification)
3. **Class-specific memory banks** (robust statistics per class)
4. **Independent optimization** (each NRAM adapts to its class)

**Expected improvements over UnifiedTTANRAM:**
- Better variance estimation
- More targeted noise suppression
- Higher accuracy on both clean and noisy data